In [10]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import time
from tqdm import tqdm_notebook
from sklearn import tree
from sklearn import svm
from sklearn import metrics

CLASS_NAMES=['bass', 'brass', 'flute', 'guitar', 
             'keyboard', 'mallet', 'organ', 'reed', 
             'string', 'synth_lead', 'vocal']

SOURCE_NAMES=['acoustic', 'electronic', 'synthetic']
DATA_GROUPS=['test', 'valid', 'train']
FEATURE_LIST=['y_harmonic', 'y_percussive', 'chroma_cens', 'mfcc','mel_spec', 'spec_contrast']

def getDataset(dataGroup, source, target, other):#, instrument, source):
    
    new_dir='Dataset/nsynth-'+dataGroup+'/audio/'     #set the audio directory (test, train, etc)
    dataframe_raw = pd.read_json(path_or_buf='Dataset/nsynth-'+dataGroup+'/examples.json', orient='index') #read all instruments from examples.json
    dataframe_specific = dataframe_raw.loc[(dataframe_raw['instrument_family_str'] == target) | (dataframe_raw['instrument_family_str'] == other)]           #narrow down by family (strings, etc)
    dataframe_specific = dataframe_specific.loc[dataframe_specific['instrument_source_str'] == source]     #narrow down by source (acoustic, etc)

   
    Y = dataframe_specific.instrument_family_str.replace(to_replace=[other, target], value=[0, 1])
    filenames = dataframe_specific.index.tolist()     #get filenames from our dataframe, put into list
    
    dictionary = {}
    for file in tqdm_notebook(filenames):             #for all files in filenames. Also,  tqdm is a loading bar
       # print(new_dir)
       # print(file)
       # print('.wav')
       # print(new_dir+file+'.wav')
        features = feature_extract((new_dir+file+'.wav')) #specify directory, file, then add .wav. we will perform feature_extract with the file
        dictionary[file] = features                       #make dictionary using file as rows - features as columns
    featureDf = pd.DataFrame.from_dict(dictionary, orient='index', #turn into dataframe
                                       columns=FEATURE_LIST)
    featureFinal = pd.concat([dataframe_specific, featureDf], axis=1, sort=False)

    featureFinal['targetValue'] = Y
    return featureFinal #returns dataframe of features

def feature_extract(file):
    y, sr = librosa.load(file, sr=None)
    
    hop_length = 512
    
    # Separate harmonics and percussives into two waveforms
    y_harmonic, y_percussive = librosa.effects.hpss(y)    

    #Mel Spectrogram
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, 
                                                 fmax = 8000)
    #Mel-Frequency Cepstral Coefficients (MFCC) features from the raw signal
    mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)
    
    #Spectral Contrast
    spec_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    
    chroma_cens = librosa.feature.chroma_cens(y=y, sr=sr)
    
    y_harmonic = np.mean(y_harmonic)
    y_percussive = np.mean(y_percussive)
    mel_spec = np.mean(mel_spec, axis=1)
    mfcc = np.mean(mfcc, axis =1)
    chroma_cens = np.mean(chroma_cens, axis=1)
    spec_contrast = np.mean(spec_contrast, axis=1)
    
    return [y_harmonic, y_percussive, chroma_cens, mfcc, mel_spec, 
            spec_contrast]

In [13]:
def runSVM():
    dataGroup = DATA_GROUPS[0]    #SET IF YOU WANT TEST, TRAIN, OR VALID (IF YOU HAVE IT)
    instrumentTarget = CLASS_NAMES[8]   #SET THE INSTRUMENT YOU WANT
    source = SOURCE_NAMES[0]      #SET ACOUSTIC, ELECTRONIC, SYNTHETIC
    instrumentOther = CLASS_NAMES[4]

    test = getDataset(dataGroup, source, instrumentTarget, instrumentOther)   #GETS DATAFRAME WITH FEATURES EXTRACTED BY AVERY

    testX = test[FEATURE_LIST[0]] 
    testY = test['targetValue']

    print(test.targetValue)

    
    
    
    dataGroup = DATA_GROUPS[2]    #SET IF YOU WANT TEST, TRAIN, OR VALID (IF YOU HAVE IT)
    instrumentTarget = CLASS_NAMES[8]   #SET THE INSTRUMENT YOU WANT
    source = SOURCE_NAMES[0]      #SET ACOUSTIC, ELECTRONIC, SYNTHETIC
    instrumentOther = CLASS_NAMES[4]

    train = getDataset(dataGroup, source, instrumentTarget, instrumentOther)
    trainX = train[FEATURE_LIST[0]] 
    trainY = train['targetValue']

    clf = svm.SVC(kernel='linear')

    clf.fit(trainX, trainY)

    prediction = clf.predict(testX)
    accuracy = metrics.accuracy_score(testY, prediction)

    print("Accuracy: ", accuracy)
    return(prediction, accuracy)

SyntaxError: invalid syntax (<ipython-input-12-cb7df813d005>, line 5)

In [14]:
prediction,accuracy = runSVM()


keyboard_acoustic_004-022-050    0
keyboard_acoustic_004-025-100    0
keyboard_acoustic_004-026-050    0
keyboard_acoustic_004-026-100    0
keyboard_acoustic_004-027-050    0
                                ..
string_acoustic_080-049-127      1
string_acoustic_080-050-075      1
string_acoustic_080-051-127      1
string_acoustic_080-053-050      1
string_acoustic_080-054-050      1
Name: targetValue, Length: 425, dtype: int64


KeyboardInterrupt: 